In [ ]:
import os
import numpy as np
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize
import h5py
import glob
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

class LoadAndNormalizeImages(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()

    def load_and_process_image(self, file_path):
        image = nib.load(file_path).get_fdata()
        image = self.scaler.fit_transform(image.reshape(-1, image.shape[-1])).reshape(image.shape)
        return image

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        t1_path, t2_path, t1ce_path, flair_path = X
        t1 = self.load_and_process_image(t1_path)
        t2 = self.load_and_process_image(t2_path)
        t1ce = self.load_and_process_image(t1ce_path)
        flair = self.load_and_process_image(flair_path)
        return np.stack([t1, t1ce, t2, flair], axis=3)

class ResizeImages(BaseEstimator, TransformerMixin):
    def __init__(self, target_shape=(128, 128, 128)):
        self.target_shape = target_shape

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return resize(X, self.target_shape, mode='constant', anti_aliasing=True)

class ResizeMasks(BaseEstimator, TransformerMixin):
    def __init__(self, target_shape=(128, 128, 128)):
        self.target_shape = target_shape

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        mask = resize(X, self.target_shape, mode='constant', anti_aliasing=False, preserve_range=True).astype(np.uint8)
        return to_categorical(mask, num_classes=4)

def save_patient_data(images, mask, output_path, patient_id):
    output_file = os.path.join(output_path, f'patient_{patient_id}.h5')
    with h5py.File(output_file, 'w') as hf:
        hf.create_dataset('images', data=images, compression='gzip')
        hf.create_dataset('masks', data=mask, compression='gzip')
    print(f"Imagenes y mascaras guardadas con numero de paciente {patient_id} como HDF5")

def create_pipeline(target_shape):
    return Pipeline([
        ('load_and_normalize_images', LoadAndNormalizeImages()),
        ('resize_images', ResizeImages(target_shape=target_shape)),
    ])

def process_data_folder(data_folder, pipeline, output_path):
    t1_list = sorted(glob.glob(os.path.join(data_folder, '*/*t1n.nii.gz')))
    t2_list = sorted(glob.glob(os.path.join(data_folder, '*/*t2W.nii.gz')))
    t1ce_list = sorted(glob.glob(os.path.join(data_folder, '*/*t1c.nii.gz')))
    flair_list = sorted(glob.glob(os.path.join(data_folder, '*/*t2f.nii.gz')))
    mask_list = sorted(glob.glob(os.path.join(data_folder, '*/*seg.nii.gz')))

    for img in range(len(t1ce_list)):
        print(f"Now preparing image and masks number: {img}")
        image_paths = (t1_list[img], t2_list[img], t1ce_list[img], flair_list[img])
        images = pipeline.transform(image_paths)
        mask = nib.load(mask_list[img]).get_fdata().astype(np.uint8)
        mask_resized = ResizeMasks(target_shape=pipeline.named_steps['resize_images'].target_shape).transform(mask)
        save_patient_data(images, mask_resized, output_path, img)

if __name__ == "__main__":
    data_folder = 'GLIOMA'
    target_shape = (128, 128, 128)
    output_path = 'C:/Users/lcres/Desktop/modelo'

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    pipeline = create_pipeline(target_shape)
    
    # Guardar el pipeline
    joblib.dump(pipeline, 'image_processing_pipeline.pkl')
    
    # Cargar y usar el pipeline para procesar una nueva carpeta
    pipeline = joblib.load('image_processing_pipeline.pkl')
    process_data_folder(data_folder, pipeline, output_path)
